Disclaimer: This notebook is provided for self-education and data exploration and may be freely 
shared and distributed, provided that the original author is credited.  twitter@mallet_matt

This notebook provides Python scripts to allow processing of annual EIA-923 data files, which provides annual utility scale electrical production data for the United States as published by the US Energy Information Administration.

In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

Data files for import may be accessed at https://www.eia.gov/electricity/data/eia923/.  Examples are provided to access data for 2010, 2014 and 2019 to look at 10 year trends.  Data for other years of interest may be processed similarly, by importing year specific EIA-923.

In [2]:
production_2019 = pd.read_excel("EIA923_Schedules_2_3_4_5_M_12_2019_21FEB2020.xlsx",skiprows=5)
production_2014 = pd.read_excel("EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx",skiprows=5)
production_2010 = pd.read_excel("EIA923 SCHEDULES 2_3_4_5 Final 2010.xlsx",skiprows=7)

Select relevant columns from full tables

In [3]:
nameplate_columns = ['Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'Operating Year',\
            'State','County','Technology','Prime Mover','Energy Source 1','Nameplate Capacity (MW)']

In [4]:
prod_columns =['Plant Id','Plant Name', 'Operator Name','Plant State','Reported\nPrime Mover',\
       'Reported\nFuel Type Code', 'AER\nFuel Type Code','Net Generation\n(Megawatthours)','YEAR']

Format for 2010 (and prior years) has slightly different format (column names) than EIA-923 from 
2011 to present.

In [5]:
prod_2010_columns = ['Plant ID','Plant Name', 'Operator Name','State','Reported Prime Mover',\
       'Reported Fuel Type Code', 'AER Fuel Type Code','NET GENERATION (megawatthours)','Year']

In [6]:
production_2019_short = production_2019[prod_columns].copy()
production_2014_short = production_2014[prod_columns].copy()
production_2010_short = production_2010[prod_2010_columns].copy()

Calculate total US Electricity Production (MW-hrs) for 2010, 2014 and 2019.

In [7]:
prod_total_19 = production_2019_short['Net Generation\n(Megawatthours)'].sum()
prod_total_14 = production_2014_short['Net Generation\n(Megawatthours)'].sum()
prod_total_10 = production_2010_short['NET GENERATION (megawatthours)'].sum()

from decimal import Decimal
"2019: {:.2E} MW-hr".format(Decimal(str(prod_total_19))), \
"2014: {:.2E} MW-hr".format(Decimal(str(prod_total_14))), \
"2010: {:.2E} MW-hr".format(Decimal(str(prod_total_10))), \

('2019: 4.12E+9 MW-hr', '2014: 4.09E+9 MW-hr', '2010: 4.13E+9 MW-hr')

Note that utility scale electrical generation is basically flat over this 10 year period.  The 
data evaluated here is for utility scale (1 MW nameplate and greater) electrical production, which
is reported and published in EIA-923.  There is not comparable, highly reliable data for homescale 
production (expected to be primarily solar).  It is expected that both increased energy efficiency
measures (e.g. LED lighting) and home scale solar are the reason for lack of growth in utility 
scale output, because significant population growth would be expected to result in some net 
increase in demand.  The EIA provides estimates for small scale solar at:
https://www.eia.gov/electricity/annual/html/epa_01_01.html
Estimated small scale solar is significantly less than utility scale generation.

Reported Fuel type Codes taken from EIA923 File Layout

In [10]:
fuel_dict = {'NUC': 'Nuclear', 'WAT':'Hydro','GEO':'Geothermal','WND':'Wind','SUN':'Solar'}
fuel_dict.update(dict.fromkeys(['SUB','BIT','RC','WC','LIG', 'ANT','SC','SGC'],'Coal'))
fuel_dict.update(dict.fromkeys(['NG','PG'],'Natural Gas')) 
fuel_dict.update(dict.fromkeys(['WDS','BLQ','WDL'],'Wood'))
fuel_dict.update(dict.fromkeys(['OBG','OBS','OBL','AB','MSB'],'Biomass'))
fuel_dict.update(dict.fromkeys(['RFO','DFO','JF','SGP','KER','PC'],'Petroleum'))
fuel_dict.update(dict.fromkeys(['MSN','LFG','TDF', 'WO','SLW'],'Waste'))
fuel_dict.update(dict.fromkeys(['WH', 'OTH','PUR','OG', 'BFG'],'Other'))

Based on looking at data from 2010-2019, it is notable that solar generation has increased
from less than 0.1% of electrical production in 2010 to 1.7% in 2019.  Likewise, coal's share
of electrical generation has dropped from 44% to 23%, while natural gas has increased its share
from 24% to 38%.  Overall fossil production (coal and natural gas combined) has dropped from
69% to 62% of electrical generation, with all of the lost market share going to wind and solar.

In [13]:
production_2019_short['Primary Fuel'] = production_2019_short['Reported\nFuel Type Code'].map(fuel_dict)
Prod_type_summary_19 = production_2019_short.groupby(['Primary Fuel'])['Net Generation\n(Megawatthours)'].sum()
Prod_type_summary_19.sort_values(ascending=False)

Primary Fuel
Natural Gas    1.581826e+09
Coal           9.661483e+08
Nuclear        8.094093e+08
Wind           3.000710e+08
Hydro          2.684466e+08
Solar          7.223417e+07
Wood           3.985077e+07
Waste          1.949644e+07
Other          1.868743e+07
Petroleum      1.752268e+07
Geothermal     1.601059e+07
Biomass        8.421288e+06
Name: Net Generation\n(Megawatthours), dtype: float64

Calculate % breakdown of US Electricity Production

In [14]:
(Prod_type_summary_19/prod_total_19).sort_values(ascending=False)

Primary Fuel
Natural Gas    0.384120
Coal           0.234613
Nuclear        0.196552
Wind           0.072867
Hydro          0.065188
Solar          0.017541
Wood           0.009677
Waste          0.004734
Other          0.004538
Petroleum      0.004255
Geothermal     0.003888
Biomass        0.002045
Name: Net Generation\n(Megawatthours), dtype: float64

In [63]:
production_2014_short['Primary Fuel'] = production_2014_short['Reported\nFuel Type Code'].map(fuel_dict)
Prod_type_summary_14 = production_2014_short.groupby(['Primary Fuel'])['Net Generation\n(Megawatthours)'].sum()
Prod_type_summary_14.sort_values(ascending=False)

Primary Fuel
Coal           1.581710e+09
Natural Gas    1.126623e+09
Nuclear        7.971660e+08
Hydro          2.531931e+08
Wind           1.816553e+08
Wood           4.233973e+07
Petroleum      2.973544e+07
Waste          1.991889e+07
Solar          1.769103e+07
Other          1.736642e+07
Geothermal     1.587694e+07
Biomass        1.029663e+07
Name: Net Generation\n(Megawatthours), dtype: float64

In [64]:
(Prod_type_summary_14/prod_total_14).sort_values(ascending=False)

Primary Fuel
Coal           0.386386
Natural Gas    0.275215
Nuclear        0.194734
Hydro          0.061851
Wind           0.044375
Wood           0.010343
Petroleum      0.007264
Waste          0.004866
Solar          0.004322
Other          0.004242
Geothermal     0.003878
Biomass        0.002515
Name: Net Generation\n(Megawatthours), dtype: float64

In [65]:
production_2010_short['Primary Fuel'] = production_2010_short['Reported Fuel Type Code'].map(fuel_dict)
Prod_type_summary_10 = production_2010_short.groupby(['Primary Fuel'])['NET GENERATION (megawatthours)'].sum()
Prod_type_summary_10.sort_values(ascending=False)

Primary Fuel
Coal           1.847290e+09
Natural Gas    9.877243e+08
Nuclear        8.069683e+08
Hydro          2.547019e+08
Wind           9.465225e+07
Wood           3.717216e+07
Petroleum      3.656953e+07
Other          1.690359e+07
Waste          1.625023e+07
Geothermal     1.521921e+07
Biomass        1.039595e+07
Solar          1.212182e+06
Name: NET GENERATION (megawatthours), dtype: float64

In [66]:
(Prod_type_summary_10/prod_total_10).sort_values(ascending=False)

Primary Fuel
Coal           0.447821
Natural Gas    0.239445
Nuclear        0.195626
Hydro          0.061745
Wind           0.022946
Wood           0.009011
Petroleum      0.008865
Other          0.004098
Waste          0.003939
Geothermal     0.003689
Biomass        0.002520
Solar          0.000294
Name: NET GENERATION (megawatthours), dtype: float64

Get Palo Verde production data

In [8]:
nuc = production_2019_short[production_2019_short['Reported\nFuel Type Code'] == 'NUC']
nuc = nuc.sort_values(by=['Net Generation\n(Megawatthours)'], ascending = False)
nuc.groupby('Plant Name')['Net Generation\n(Megawatthours)'].sum().sort_values(ascending=False)

Plant Name
Palo Verde                            31920368.0
Browns Ferry                          29519900.0
Peach Bottom                          22294385.0
South Texas Project                   21993297.0
Oconee                                21884681.0
TalenEnergy Susquehanna               20919007.0
Braidwood Generation Station          20251140.0
Byron Generating Station              20117981.0
Vogtle                                19674300.0
Catawba                               19593607.0
McGuire                               19588255.0
LaSalle Generating Station            19435255.0
Limerick                              19345594.0
Comanche Peak                         19304710.0
Sequoyah                              18064619.0
PSEG Salem Generating Station         17910378.0
Watts Bar Nuclear Plant               17655786.0
Millstone                             16733398.0
Diablo Canyon                         16165384.0
Donald C Cook                         16157848.0
Nine Mile